In [1]:
%run LeadQualificationCrew.py
%run EmailEngagementCrew.py
%run parse_raw_op.py

Lead Qualification Crew Activated(New)!!!
Email Engagement Crew Activated (Optimized)!
Lead Qualification Crew Activated(New)!!!
Parser Initialized!!!


In [2]:
import re
import json
from crewai import Flow
from crewai.flow.flow import listen, start

from EmailEngagementCrew import email_crew
from LeadQualificationCrew import lead_crew
from parse_raw_op import parse_crew_raw 


Email Engagement Crew Activated (Optimized)!
Parser Initialized!!!


In [3]:
class AutonomousLeadIntelligenceSystem(Flow):
    @start()
    def fetch_leads(self):
        with open('lead_data.json', 'r') as f:
            leads = json.load(f)
        return leads
    
    @listen(fetch_leads)
    def score_leads(self, leads):
        self.task_outputs = lead_crew.kickoff_for_each(leads)

        all_lead_results = []
        for task_output in self.task_outputs:
            lead_result = getattr(task_output, "pydantic", None)
            if lead_result:
                all_lead_results.append(lead_result)

        lead_results_dicts = [result.dict() for result in all_lead_results]
        with open("lead_scores.json", "w", encoding="utf-8") as f:
            json.dump(lead_results_dicts, f, indent=4)

        return all_lead_results 

    @listen(score_leads)
    def filter_leads(self,all_lead_results):
        filtered = [
            lead_result for lead_result in all_lead_results
            if lead_result.lead_score.score > 70
        ]

        filtered_dicts = [lead.dict() for lead in filtered]
        with open("filtered_leads.json", "w", encoding="utf-8") as f:
            json.dump(filtered_dicts, f, indent=4)

        return filtered_dicts
    
    @listen(filter_leads)
    def write_email(self, filtered_dicts):
        self.email_results = email_crew.kickoff_for_each(filtered_dicts)

        # Extract plaintext email bodies
        final_emails = []
        for output in self.email_results:
            final_emails.append(output.raw)

        # Add email text into each lead dict
        for lead, email_text in zip(filtered_dicts, final_emails):
            lead["generated_email"] = email_text

        # Save updated leads with emails
        with open("filtered_leads.json", "w", encoding="utf-8") as f:
            json.dump(filtered_dicts, f, indent=4)

        return final_emails   

    @listen(write_email)
    def send_email(self, emails):
        print("\nEMAILS READY TO SEND:\n", emails)
        return emails 

print("Pipeline Created!!!")

Pipeline Created!!!


In [4]:
flow = AutonomousLeadIntelligenceSystem()
# flow.plot()

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: AutonomousLeadIntelligenceSystem                                                                         │
│  ID: fb2f5eeb-b55f-41b2-9777-6ac2004a691b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [5]:
import nest_asyncio
nest_asyncio.apply()

emails = await flow.kickoff_async()

Flow started with ID: fb2f5eeb-b55f-41b2-9777-6ac2004a691b


Output()

Flow started with ID: fb2f5eeb-b55f-41b2-9777-6ac2004a691b


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Data Specialist                                                                                    │
│                                                                                                                 │
│  Task: Collect and analyze the following information about the lead.                                            │
│  Personal Information:                                                                                          │
│    - Name                                                                                                       │
│    - Job Title                                                                                                  │
│    - Role Relevance (0â€“10)                                                                                    │
│    - Professional Background (optional)                                                                         │
│                                                                                                                 │
│  Company Information:                                                                                           │
│    - Company Name                                                                                               │
│    - Industry                                                                                                   │
│    - Company Size (employees)                                                                                   │
│    - Revenue (optional)                                                                                         │
│    - Market Presence (0â€“10)                                                                                   │
│                                                                                                                 │
│  Use this provided lead data: "{'name': 'Satya Nadella', 'job_title': 'CEO', 'company': 'Microsoft', 'email':   │
│  'satya.office@microsoft.com', 'use_case': 'Exploring autonomous AI agents to optimize enterprise-scale data    │
│  workflows.'}"                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Data Specialist                                                                                    │
│                                                                                                                 │
│  Thought: I need to gather detailed information about the lead, Satya Nadella, and his company Microsoft.       │
│  Since I need to find specific information on Microsoft, particularly regarding its industry, size, and market  │
│  presence, I will start by searching for the company details.                                                   │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Scorer and Validator                                                                               │
│                                                                                                                 │
│  Task: Aggregate all collected data and calculate the final lead score (0â€“100).                               │
│  Scoring factors:                                                                                               │
│    - Role Relevance                                                                                             │
│    - Company Size                                                                                               │
│    - Market Presence                                                                                            │
│    - Cultural Fit                                                                                               │
│                                                                                                                 │
│  Required steps:                                                                                                │
│    - Calculate score                                                                                            │
│    - List scoring criteria                                                                                      │
│    - Validate score accuracy                                                                                    │
│    - Produce a final structured JSON/Pydantic report                                                            │
│                                                                                                                 │
│  Use this provided lead data: "{'name': 'Daniel Ek', 'job_title': 'CEO', 'company': 'Spotify', 'email':         │
│  'dek@spotify-execmail.com', 'use_case': 'Considering lightweight AI assistants for content workflow            │
│  management and user recommendation refinement.'}"                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Scorer and Validator                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "personal_info": {                                                                                           │
│      "name": "Daniel Ek",                                                                                       │
│      "job_title": "CEO",                                                                                        │
│      "role_relevance": 10,                                                                                      │
│      "professional_background": "Daniel Ek is the co-founder and CEO of Spotify, a prominent figure in the      │
│  music streaming industry since its inception in 2006."                                                         │
│    },                                                                                                           │
│    "company_info": {                                                                                            │
│      "company_name": "Spotify Technology S.A.",                                                                 │
│      "industry": "Audio Streaming and Podcasting",                                                              │
│      "company_size": 7323,                                                                                      │
│      "revenue": 15670,                                                                                          │
│      "market_presence": 10                                                                                      │
│    },                                                                                                           │
│    "lead_score": {                                                                                              │
│      "score": 95,                                                                                               │
│      "scoring_criteria": [                                                                                      │
│        "Role Relevance: 10/10 - CEO level with direct influence on purchasing decisions.",                      │
│        "Company Size: 10/10 - Large established company with adequate budget for AI solutions.",                │
│        "Market Presence: 10/10 - Strong global brand and recognition in the audio streaming industry.",         │
│        "Cultural Fit: 7/10 - Good alignment with innovation and ethical practices; slight discrepancies in      │
│  industry focus."                                                                                               │
│      ],                                                                                                         │
│      "validation_notes": "Score calculations verified against scoring criteria for accuracy and consistency."   │
│    }                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

EMAILS READY TO SEND:

['Your expertise at Microsoft aligns perfectly with LuminaSyn’s AI-driven platform. Ready to enhance enterprise 
solutions? Let’s schedule a quick call to discuss how we can tailor our innovations for your teams. Click here to 
book a time that works for you! Don’t miss out on transforming your approach to learning and efficiency.', "I see 
your interest in our AI-driven adaptive learning platform at LuminaSyn Industries. Ready to transform your team's 
learning? Click here to schedule a quick demo and discover tailored solutions. Don't miss out on maximizing your 
team's potential—let's chat! What time works for you?", "Mary, let's unlock your team's potential with LuminaSyn's 
AI-driven platform. Schedule a quick call to discuss how we can customize our solution for GM's unique challenges. 
Click here to pick a time that suits you: [Insert Link]. Don't miss out on transforming your enterprise 
performance—let's connect today!", "Tim, your leadership at Apple Inc. aligns perfectly with LuminaSyn's AI-driven 
adaptive learning platform. Ready to elevate your enterprise teams’ performance? Click here to schedule a quick 
call this week and explore how we can drive success together. Don’t miss out on this opportunity—let’s connect!", 
"Daniel, your leadership at Spotify is inspiring. LuminaSyn’s AI-driven platform can elevate your team's 
performance. Ready to explore how we can customize our solution for Spotify? Click here to schedule a quick chat or
reply with your availability. Let’s innovate together!"]

In [25]:
with open("final_output_emails.json", "w") as f:
    emails_dict = {}
    for i, email in enumerate(emails):
        emails_dict[f"Email {i+1} :"] = email
    json.dump(emails_dict, f)